In [ ]:
!wget "https://www.ncbi.nlm.nih.gov/geo/download/?acc=GSE150062&format=file&file=GSE150062%5Fseurat%2Erds%2Egz" -O GSE150062_seurat.rds.gz

In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

%mkdir -p /content/drive/MyDrive/GSE150062
%cd /content/drive/MyDrive/GSE150062

/content/drive/MyDrive/GSE150062


In [2]:
%pip install scanpy python-dotenv -q

import os
import tarfile
import json
import gzip

import scanpy as sc
import anndata as ad
import pandas as pd

from dotenv import load_dotenv
load_dotenv(dotenv_path='/content/drive/MyDrive/.gse.env')
COLS = json.loads(os.environ.get('OBS_COLS'))
ORGANISMS = json.loads(os.environ.get('ORGANISMS'))

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.4/124.4 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 kB 6.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 11.3 MB/s eta 0:00:00


In [ ]:

# %%bash
# for feat in "features.tsv.gz" "barcodes.tsv.gz"; do
#     if [ ! -f "$feat" ]; then
#         echo "${feat%.gz} not gzipped"
#         gzip "${feat%.gz}"
#     else
#         echo "${feat%.gz} checked"
#     fi
# done

In [ ]:
# %%bash
# mkdir -p out
# mv *.gz out/

In [ ]:
adata = ad.read_mtx('out/matrix.mtx.gz')
adata

AnnData object with n_obs × n_vars = 60497 × 78393

In [ ]:
obs = pd.read_csv("./GSE150062_obs.csv", index_col=0)
obs.head()

,guide,perturbation_name,condition,cell_type,organism,crispr_type,cancer_type
AAACGAATCAGACCGC-1_1,sgLH02412_1,LH02412,Neural differentiation,iPSCs (induced pluripotent stem cells),Humans (Homo sapiens),CRISPRi,non-cancer
AAAGGGCCAGAGAATT-1_1,sgIMMP1L_1,IMMP1L,Neural differentiation,iPSCs (induced pluripotent stem cells),Humans (Homo sapiens),CRISPRi,non-cancer
AAAGGGCCATGAGAAT-1_1,sgPCGF2_1,PCGF2,Neural differentiation,iPSCs (induced pluripotent stem cells),Humans (Homo sapiens),CRISPRi,non-cancer
AAAGTCCCATGGCACC-1_1,sgWNT5B_1,WNT5B,Neural differentiation,iPSCs (induced pluripotent stem cells),Humans (Homo sapiens),CRISPRi,non-cancer
AAAGTCCTCGAGTACT-1_1,sgAMBRA1_2,AMBRA1,Neural differentiation,iPSCs (induced pluripotent stem cells),Humans (Homo sapiens),CRISPRi,non-cancer


In [ ]:
adata = adata.T
adata.obs = obs

In [ ]:
genes = pd.read_csv('./out/features.tsv.gz', sep='\t')
genes.head()

,gene_symbol
0,"1 ""DDX11L1"""
1,"2 ""MIR1302-11"""
2,"3 ""OR4G4P"""
3,"4 ""OR4G11P"""
4,"5 ""OR4F5"""


In [ ]:
adata.var.index = genes['gene_symbol'].str.extract(r'"([^"]*)"')[0]
adata.var.index.name = 'gene_symbol'
adata

AnnData object with n_obs × n_vars = 78393 × 60497
    obs: 'guide', 'perturbation_name', 'condition', 'cell_type', 'organism', 'crispr_type', 'cancer_type'

In [ ]:
adata.var_names, adata.obs_names

(Index(['DDX11L1', 'MIR1302-11', 'OR4G4P', 'OR4G11P', 'OR4F5', 'CICP27',
        'RP11-34P13.9', 'RP4-669L17.10', 'LH00001', 'LH00002',
        ...
        'MT-CO2', 'MT-ATP8', 'MT-ATP6', 'MT-CO3', 'MT-ND3', 'MT-ND4L', 'MT-ND4',
        'MT-ND5', 'MT-CYB', 'MT-ND6'],
       dtype='object', name='gene_symbol', length=60497),
 Index(['AAACGAATCAGACCGC-1_1', 'AAAGGGCCAGAGAATT-1_1', 'AAAGGGCCATGAGAAT-1_1',
        'AAAGTCCCATGGCACC-1_1', 'AAAGTCCTCGAGTACT-1_1', 'AACAAAGGTAGCTTAC-1_1',
        'AACAAAGTCCCTGTTG-1_1', 'AACAACCCAAGCGCAA-1_1', 'AACAGGGCACGGTGAA-1_1',
        'AACAGGGGTACGAAAT-1_1',
        ...
        'TTCATTGCAGGTGTTT-1_6', 'TTCCAATAGGGTACAC-1_6', 'TTCCTTCTCCCTCTCC-1_6',
        'TTGAACGAGCAAGTCG-1_6', 'TTGACCCAGCTGGCTC-1_6', 'TTGATGGGTTCCAGGC-1_6',
        'TTGGGCGAGGCTGAAC-1_6', 'TTGTTTGCACTAGGCC-1_6', 'TTGTTTGTCGCCTTGT-1_6',
        'TTTATGCGTCTGATAC-1_6'],
       dtype='object', length=78393))

In [ ]:
adata.obs['perturbation_name'] = adata.obs['perturbation_name'].apply(lambda x: 'non-targeting' if x == 'Non-Targeting' else x)
adata.obs['perturbation_name'].value_counts()

perturbation_name
non-targeting    3423
SUFU             1097
MED24             709
SOX11             637
AMBRA1            627
                 ... 
LH03635            54
LH01001            50
LH08564            34
LH01249            28
LH17063            24
Name: count, Length: 241, dtype: int64

In [ ]:
adata.write_h5ad('./GSE150062_merged.h5ad')

In [ ]:
missing = [col for col in COLS if col not in adata.obs]
print(f"Missing following columns: \n {missing}")

if 'organism' in adata.obs_keys():
    assert adata.obs['organism'].isin(ORGANISMS).all(), "Invalid organism naming"

Missing following columns: 
 []


In [ ]:
adata = sc.read_h5ad('./GSE150062_merged.h5ad')
obs = pd.read_csv("./GSE150062_obs_all.csv", index_col=0)
adata.obs = obs
adata.obs['perturbation_name'] = adata.obs['perturbation_name'].apply(lambda x: 'non-targeting' if x == 'Non-Targeting' else x)

In [ ]:
adata.obs.head()

,orig.ident,nCount_RNA,nFeature_RNA,nCount_grna,nFeature_grna,sample,condition,UMI_count,z_score,guide,...,low_res_cluster,trajectory,library,primary_pheno,diff_pheno,diff_hit_direction,cell_type,organism,crispr_type,cancer_type
AAACGAATCAGACCGC-1_1,diff_1,22521,5346,69,19,diff_1,targeting,45,1.088537,sgLH02412_1,...,9,Cell Cycle,lncRNA,NaN,-0.235598,Non-hit,iPSCs (induced pluripotent stem cells),Humans (Homo sapiens),CRISPRi,non-cancer
AAAGGGCCAGAGAATT-1_1,diff_1,15066,3902,426,11,diff_1,targeting,407,1.269091,sgIMMP1L_1,...,4,NSC,Coding,Differentiation,1.925859,Positive Hit,iPSCs (induced pluripotent stem cells),Humans (Homo sapiens),CRISPRi,non-cancer
AAAGGGCCATGAGAAT-1_1,diff_1,20844,5095,670,16,diff_1,targeting,649,0.908249,sgPCGF2_1,...,4,NSC,Coding,Differentiation,2.117745,Positive Hit,iPSCs (induced pluripotent stem cells),Humans (Homo sapiens),CRISPRi,non-cancer
AAAGTCCCATGGCACC-1_1,diff_1,14015,4028,820,13,diff_1,targeting,805,1.661506,sgWNT5B_1,...,9,Cell Cycle,Coding,NaN,-0.058791,Non-hit,iPSCs (induced pluripotent stem cells),Humans (Homo sapiens),CRISPRi,non-cancer
AAAGTCCTCGAGTACT-1_1,diff_1,34393,6566,535,14,diff_1,targeting,518,0.757062,sgAMBRA1_2,...,7,Non-NSC,Coding,Dual,0.877920,Positive Hit,iPSCs (induced pluripotent stem cells),Humans (Homo sapiens),CRISPRi,non-cancer


In [ ]:
adata.obs.loc[adata.obs['primary_pheno'] == 'Control', 'perturbation_name'].unique()

array(['non-targeting'], dtype=object)

In [ ]:
adata.obs.loc[adata.obs['primary_pheno'].isna(), 'perturbation_name'].unique()

array(['LH02412', 'WNT5B', 'LH10263', 'LH03957', 'LH06838', 'LH03970',
       'SPRYD7', 'ARL6IP5', 'LH14698', 'LH05664', 'TMEM91', 'P4HB',
       'LH15554', 'LH09697', 'LH05070', 'LH01085', 'LH05817', 'LH13623',
       'LH02843', 'SDC3', 'LH06255', 'LH14166', 'CEP95', 'LH03298',
       'ZNF551', 'LH16684', 'LH14300', 'LH11067', 'LH04266', 'LH05273',
       'LH15443', 'LH13995', 'LH02347', 'LH05311', 'LH14945', 'LH07082',
       'LH11113', 'LH08334', 'UBXN2A', 'TESK2', 'LH11697', 'LH01622',
       'ARHGAP31', 'MRPL27', 'LH08553', 'C1orf74', 'PPFIA3', 'LH11698',
       'LH06021', 'COA6', 'LH09173', 'LH02756', 'LH01957', 'LH16669',
       'LH09049', 'LH16361', 'GGT7', 'KDM4C', 'LH09192', 'LH13281',
       'NABP2', 'LH01209', 'LH12157', 'RPP14', 'GTPBP8'], dtype=object)

In [ ]:
adata.obs['primary_pheno'].value_counts()

primary_pheno
Differentiation    42936
Dual                9868
Control             3423
Proliferation        477
Name: count, dtype: int64

In [ ]:
adata.obs['condition'] = adata.obs['primary_pheno'].apply(lambda x: x if not pd.isna(x) else 'Other')
adata.obs['condition'].value_counts()

condition
Differentiation    42936
Other              21689
Dual                9868
Control             3423
Proliferation        477
Name: count, dtype: int64

In [ ]:
adata.write_h5ad('./GSE150062_merged.h5ad')

In [3]:
adata = sc.read_h5ad('./GSE150062_merged.h5ad')
adata.obs['perturbation_name'].value_counts()

perturbation_name
non-targeting    3423
SUFU             1097
MED24             709
SOX11             637
AMBRA1            627
                 ... 
LH03635            54
LH01001            50
LH08564            34
LH01249            28
LH17063            24
Name: count, Length: 241, dtype: int64